# Типы данных

## Подготовка

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
DROP DATABASE IF EXISTS my_db
CASCADE
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
CREATE DATABASE my_db
"

## Строковые типы данных

### Создание таблицы с колонками строкового типа

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
CREATE TABLE my_text_type_table(
  string_value STRING,
  varchar15_value VARCHAR(15),
  char15_value CHAR(15)
)
"

### Добавление строковых данных

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
INSERT INTO my_text_type_table
WITH records AS (
    SELECT 'Hello, World!' text UNION
    SELECT 'Привет, мир!' text UNION
    SELECT 'Пока' text
)
SELECT text
     , cast(text as VARCHAR(15))
     , cast(text as CHAR(15))
  FROM records
"

### Проверка вставленных строковых данных

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
SELECT *
  FROM my_text_type_table
"

### Расследование проблем с отображением строковых данных

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
SELECT string_value str
     , length(string_value) str_length
     , varchar15_value varchar15
     , length(varchar15_value) varchar15_length
     , char15_value char15
     , length(char15_value) char15_length
  FROM my_text_type_table
"

> **Cимволы на кириллице занимают два байта на символ**

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
WITH chars AS (
    SELECT 'П' c
)
SELECT c original
     , cast(c as CHAR(1)) char1
     , cast(c as CHAR(2)) char2
     , cast(c as VARCHAR(1)) varchar1
     , cast(c as VARCHAR(2)) varchar2
  FROM chars
"

Проблема с отображением данных заключается в том, что 15 байт не хватает для хранения фразы "Привет, мир!", и Impala просто обрезает фразу по границе доступного буффера для `VARCHAR` и `CHAR` типов данных.

> `VARCHAR` и `CHAR` сохранят только ту часть данных, которая помещается в буффер, оставшаяся часть будет игнорироваться

### Максимальный размер `CHAR`

Формат данных `CHAR` может хранить не больше 255 байт:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SELECT cast('П' as CHAR(256))
" || true

Подобного ограничения нет для `VARCHAR`:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SELECT cast('П' as VARCHAR(256))
" || true

### Выводы

1. Тип `STRING` является универсальным при работе со строковыми данными;
1. Типы `VARCHAR` и `CHAR` желательно избегать в силу их молчаливого поведения при сохранении данных, превышающих выделенный буффер;
1. Строковые типы данных работают с байтами, а не с символами.

### Функции для работы со строковыми типами данных

#### Длина строки

Вычисление длины текстовых данных можно выполнить при помощи функции `LENGTH`:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
WITH chars AS (
    SELECT 'Hello' c
)
SELECT c
     , LENGTH(c) content_length
  FROM chars
"

При использовании `CHAR` строка дополняется пробелами справа, чтобы заполнить выделенный буффер, поэтому для `CHAR` существует две функции вычисления длины:

1. `LENGTH` - длина без пробелов,
1. `CHAR_LENGTH` - длина с пробелами.

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
WITH chars AS (
    SELECT cast('hello' as CHAR(10)) c_no_ws UNION ALL
    SELECT cast('hello ' as CHAR(10)) UNION ALL
    SELECT cast(' hello' as CHAR(10))
)
SELECT c_no_ws
     , LENGTH(c_no_ws) content_length
     , CHAR_LENGTH(c_no_ws) full_length
  FROM chars
"

#### Соединение строк

Соединить строки можно при помощи:

- функции `CONCAT`,
- оператора `||`.

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
WITH words AS (
    SELECT 'Hello' hello
)
SELECT CONCAT(hello, ', ', 'World', '!') concat_func
     , hello || ', ' || 'World' || '!' concat_operator
  FROM words;
"

#### Соединение строк с разделителем

Функция `concat_ws` позволяет указать разделитель при соединении строк

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
WITH words AS (
    SELECT 'Hello' hello
)
SELECT CONCAT_WS('|', hello, ', ', 'World', '!') concat_ws_bar
     , CONCAT_WS('<-РАЗДЕЛИТЕЛЬ->', hello, ', ', 'World', '!') concat_ws_text
  FROM words;
"

#### Выбор подстроки

Функция `SUBSTR` позволяет выбрать из строки подстроку указанной длины, начиная с указанной позиции:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
WITH words AS (
    SELECT 'Hello, World!' hello_world
)
SELECT hello_world
     , SUBSTR(hello_world, LENGTH('Hello,'), 7) substr_func
     , SUBSTR(hello_world, LENGTH('Hello,'), 500) substr_func_length_max
     , SUBSTR(hello_world, LENGTH('Hello,')) substr_func_length_max
  FROM words;
"

#### Поиск подстроки

Функция `INSTR` позволяет найти индекс в строке, с которого начинается подстрока:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
WITH words AS (
    SELECT 'Hello, World' hello_world
)
SELECT hello_world
     , INSTR(hello_world, 'o') instr_first_index
     , INSTR(hello_world, 'o', 6) instr_second_index
     , INSTR(hello_world, 'o', 1, 2) instr_second_occurrence_index
     , INSTR(hello_world, 'xxx', 1, 2) instr_second_occurrence_index
  FROM words;
"

#### Заполнение строки

Функции `RPAD` и `LPAD` позволяет добавить указанную строку в оригинальную строку несколько раз, чтобы общая длина итоговой строки стала равной указанной длине:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
WITH words AS (
    SELECT 'Hello' hello
)
SELECT hello
     , RPAD(hello, 20, '>/-+') rpad_gt
     , LPAD(hello, 20, '+-/<') lpad_lt
  FROM words;
"

#### Сокращение строки

Функции `TRIM`, `RRIM` и `LTRIM` удалить лидирующие, хвостовые и пробелы вокруг строки:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
WITH words AS (
    SELECT 'Hello' hello
         , '    Hello    ' hello_ws
         , '\n\tHello\n\t    ' hello_lf_tab
)
SELECT hello_ws
     , '<' || TRIM(hello_ws) || '>' hello_ws_trimmed
     , '<' || LTRIM(hello_ws) || '>' hello_ws_ltrimmed
     , '<' || RTRIM(hello_ws) || '>' hello_ws_rtrimmed
  FROM words;
"

Также можно указать, какие символы вместо пробелов необходимо удалить:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
WITH words AS (
    SELECT 'Hello' hello
)
SELECT '<' || hello || '>' hello
     , '<' || LTRIM(hello, 'He') || '>' hello_ltrimmed_he
     , '<' || RTRIM(hello, 'ol') || '>' hello_rtrimmed_ol
  FROM words;
"

Символы табуляции (`\t`) и перевода строки (`\n`) не являются пробельными с точки зрения Impala, поэтому необходимо явно их удалить при необходимости:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
WITH words AS (
    SELECT '\n  \tHello\n\t    ' hello_lf_tab
)
SELECT '<' || hello_lf_tab || '>' c
     , '<' || TRIM(hello_lf_tab) || '>' hello_lf_tab_trimmed
     , '<' || LTRIM(hello_lf_tab, '\n\t ') || '>' hello_lf_tab_trimmed
     , '<' || RTRIM(hello_lf_tab, '\n\t ') || '>' hello_lf_tab_trimmed
  FROM words;
"

#### Изменение регистра

Функции `LOWER` и `UPPER` используются для перевода всех символов в нижний или верхний регистр соответственно:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
WITH words AS (
    SELECT 'HeLlo, WorlD!' hello_world UNION
    SELECT 'ПриВет, МиР!'
)
SELECT hello_world
     , LOWER(hello_world) lower_hello
     , UPPER(hello_world) upper_hello
  FROM words;
"

#### Обращение порядка символов строки

Функция `REVERSE` меняет порядок следования символов в строке: первый становится последним и последний первым:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
WITH words AS (
    SELECT '- Hello, World!' hello_world
)
SELECT hello_world
     , REVERSE(hello_world) reversed
  FROM words;
"

### Поддержка Unicode

Impala работает с текстом как с массивом байт, где один символ занимает один байт. Если использовать текст с символами не из ASCII таблицы (многобайтовые кодировки, UTF-8), то могут возникнуть проблемы:

Следующий запрос показывает, что длина кириллического символа `п`, занимает 2 байта:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
WITH chars AS (
    SELECT 'п' str
         , CAST('п' AS VARCHAR(5)) vchr
         , CAST('п' AS CHAR(5)) chr
)
SELECT LENGTH(str)
     , CHAR_LENGTH(str)
     , LENGTH(vchr)
     , CHAR_LENGTH(vchr)
     , LENGTH(chr)
     , CHAR_LENGTH(chr)
  FROM chars
"

Для корректной работы с символами, находящимися за пределами ASCII таблицы необходимо активировать режим UTF8, при этом тип данных `CHAR` все равно работает некорректно:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
SET UTF8_MODE=true;

WITH chars AS (
    SELECT 'п' str
         , CAST('п' AS VARCHAR(5)) vchr
         , CAST('п' AS CHAR(5)) chr
)
SELECT LENGTH(str)
     , CHAR_LENGTH(str)
     , LENGTH(vchr)
     , CHAR_LENGTH(vchr)
     , LENGTH(chr)
     , CHAR_LENGTH(chr)
  FROM chars
"

Функция `SUBSTR` также рабоатет с байтами по умолчанию:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
SET UTF8_MODE=false; -- по умолчанию

WITH chars AS (
    SELECT 'привет' str
         , CAST('привет' AS VARCHAR(20)) vchr
         , CAST('привет' AS CHAR(20)) chr
)
SELECT str
     , SUBSTR(str, 2) sub_str_2
     , SUBSTR(str, 3) sub_str_3
     , SUBSTR(vchr, 2) sub_vchr_2
     , SUBSTR(vchr, 3) sub_vchr_3
     , SUBSTR(chr, 2) sub_chr_2
     , SUBSTR(chr, 3) sub_chr_3
  FROM chars
"

Активация режима UTF8 позволяет заставляет функцию `SUBSTR` работать с символами:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
SET UTF8_MODE=true;

WITH chars AS (
    SELECT 'привет' str
         , CAST('привет' AS VARCHAR(20)) vchr
         , CAST('привет' AS CHAR(20)) chr
)
SELECT str
     , SUBSTR(str, 2) sub_str_2
     , SUBSTR(str, 3) sub_str_3
     , SUBSTR(vchr, 2) sub_vchr_2
     , SUBSTR(vchr, 3) sub_vchr_3
     , SUBSTR(chr, 2) sub_chr_2
     , SUBSTR(chr, 3) sub_chr_3
  FROM chars
"

Результат вызова функции `REVERSE` на строках с многобайтовыми кодировками ставновится совсем непонятным, т.к. функция меняет порядок байт даже в рамках одного символа:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
SET UTF8_MODE=false;

WITH chars AS (
    SELECT '-привет--' str
         , CAST('-привет--' AS VARCHAR(20)) vchr
         , CAST('-привет--' AS CHAR(20)) chr
)
SELECT str
     , REVERSE(str) sub_str
     , REVERSE(vchr) sub_vchr
     , REVERSE(chr) sub_chr
  FROM chars
"

Активация режима UTF8 позволяет заставляет функцию `REVERSE` исправляет ситуацию:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
SET UTF8_MODE=true;

WITH chars AS (
    SELECT '-привет--' str
         , CAST('-привет--' AS VARCHAR(20)) vchr
         , CAST('-привет--' AS CHAR(20)) chr
)
SELECT str
     , REVERSE(str) rev_str
     , REVERSE(vchr) rev_vchr
     , REVERSE(chr) rev_chr
  FROM chars
"

Функция `INSTR` не работает корректно с символами за пределами ASCII таблицы:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
SET UTF8_MODE=false;

WITH chars AS (
    SELECT 'привет' str
         , CAST('привет' AS VARCHAR(20)) vchr
         , CAST('привет' AS CHAR(20)) chr
)
SELECT str
     , INSTR(str, 'вет') instr_str
     , INSTR(vchr, 'вет') instr_vchr
     , INSTR(chr, 'вет') instr_chr
  FROM chars
"

даже с учетом активации режима UTF8:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
SET UTF8_MODE=true;

WITH chars AS (
    SELECT 'привет' str
         , CAST('привет' AS VARCHAR(20)) vchr
         , CAST('привет' AS CHAR(20)) chr
)
SELECT str
     , INSTR(str, 'вет') instr_str
     , INSTR(vchr, 'вет') instr_vchr
     , INSTR(chr, 'вет') instr_chr
  FROM chars
"

Apache Impala предупреждает, что активация UTF8 может снизить производительность запросов, поэтому необходимо активировать режим UTF8 по необходимости.

## Числовые типы данных

В Impala существуют следующие числовые типы данных:

| Тип     | Название   | Размер  | Область определения             |
|---------|------------|---------|---------------------------------|
| Целый   | `TYNIINT`  | 1 байт  | [-128; 127]                     |
| Целый   | `SMALLINT` | 2 байта | [$-2^{15}$; $2^{15} - 1$]       |
| Целый   | `INT`      | 4 байта | [$-2^{31}$; $2^{31} - 1$]       |
| Целый   | `BIGINT`   | 8 байт  | [$-2^{63}$; $2^{63} - 1$]       |
| Дробный (плавающая точка) | `FLOAT`    | 4 байта | [$-1.4e^{45}$; $3.4e^{38}$]     |
| Дробный (плавающая точка) | `DOUBLE`   | 8 байт  | [$-4.94e^{324}$; $1.79e^{308}$] |
| Дробный (фиксированная точка)| `DOUBLE`   | 4-16 байт  | [$-10^{38} + 1$; $10^{38} –1$]  |

Impala автоматически выбирает тип в зависимости от размера значения (обратите внимание на ошибку `ERROR: AnalysisException: Invalid type cast of...`):

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SELECT cast(1 as date)
" || true

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SELECT cast(128 as date)
" || true

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SELECT cast(32768 as date)
" || true

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SELECT cast(2147483648 as date)
" || true

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SELECT cast(9223372036854775808 as date)
" || true

Значения за пределами `DOUBLE` автоматически становятся равными бесконечности (`INF`):

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SELECT POW(10, 309)
"

### Функции для работы с чилами

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SELECT abs(-10) absolute
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SELECT ROUND(3.1415926, 2) pi
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SELECT CEIL(3.1415926) pi
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SELECT FLOOR(3.1415926) pi
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SELECT SQRT(3.1415926) pi
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SELECT RAND()
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SELECT RAND(42)
"

## Темпоральные типы данных

Для работы с датами существуют два типа данных:

| Название    | Размер  | Область определения                                                   |
|-------------|---------|-----------------------------------------------------------------------|
| `DATE`      | 4 байта | [`0001-01-01`; `9999-12-31`]                                          |
| `TIMESTAMP` | 2 байта | [`1400-01-01 00:00:00.000000000`; `9999-12-31 23:59:59.999999999`]    |

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SELECT DATE '2013-01-01' dt
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SELECT CAST('2013-01-01' as DATE) dt
     , CAST('2013-01-01     00:05:15.01' as TIMESTAMP) ts
     , CAST('2001-01-09T01:05:01.13' AS TIMESTAMP) ts_T_separator
     , CAST(60 AS TIMESTAMP) one_minute_after_start_of_epoch
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SELECT CURRENT_DATE() dt
     , CURRENT_TIMESTAMP() ts
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SELECT CAST(CURRENT_DATE() as TIMESTAMP) ts
     , CAST(CURRENT_TIMESTAMP() as DATE) dt
"

### Функции для работы с датами

#### Арифметика на датах

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SELECT DATE_ADD(CURRENT_DATE(), INTERVAL 1 MONTH) add_one_month
     , DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH) sub_one_month
     , DATEDIFF('2021-01-01', '2022-01-01')
"

#### Unix даты

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SELECT UNIX_TIMESTAMP(CURRENT_DATE()) dt
     , UNIX_TIMESTAMP(CURRENT_TIMESTAMP()) ts
"

#### Форматирование даты

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SELECT FROM_UNIXTIME(UNIX_TIMESTAMP(CURRENT_DATE()), 'yyyy/MM/dd')
"

#### Текущий таймстемп

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SELECT now()
"

#### Части даты

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SELECT YEAR(now()) y
     , MONTH(now()) m
     , DAY(now()) d
     , DAYOFWEEK(now()) dow
     , DAYOFYEAR(now()) doy
     , HOUR(now()) h
     , MINUTE(now()) m
     , SECOND(now()) s
     , MILLISECOND(now()) ms
"

#### Интервалы

Для более точной арифметики с датами можно использовать `INTERVAL`:

In [ ]:
impala-shell -i "${IMPALA_HOST}" --vertical -q "
WITH ts AS (
    SELECT now() n
)
SELECT n
     , n - INTERVAL 1 YEAR minus_year
     , n + INTERVAL 1 MONTH plus_month
     , n + INTERVAL 1 day plus_day
     , n - INTERVAL 1 hour minus_hour
     , n + INTERVAL 1 minute plus_minute
     , n - INTERVAL 1 second minus_second
     , n + INTERVAL 1 millisecond plus_millisecond
     , n - INTERVAL 1 microsecond minus_microsecond
  FROM ts
"

> Ключ `--vertical` утилиты `impala-shell` позволяет транспонировать строки и столбцы в результате

## Логический тип данных

Impala позволяет работать с логическими данными при помощи типа `BOOLEAN`:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SELECT TRUE, FALSE
"

#### Приведение типов к `BOOLEAN`

Ненулевые значения приводятся к `TRUE`:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SELECT CAST(0 AS BOOLEAN) 0_false
     , CAST(-1 AS BOOLEAN) neg_1_true
     , CAST(4321 AS BOOLEAN) pos_4321_true 
     , CAST(3.14 AS BOOLEAN) pi_true
"

Специальные маркеры `NaN` и `Inf` приводятся к `TRUE`:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SELECT CAST(CAST('nan' AS FLOAT) as BOOLEAN) nan_true
     , CAST(CAST('inf' AS FLOAT) as BOOLEAN) inf_true
"

Невозможно привести к `BOOLEAN` строки:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SELECT CAST('Hello' AS BOOLEAN)
" || true

Невозможно привести к `BOOLEAN` даты:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SELECT CAST(current_date() AS BOOLEAN)
" || true

Значения типа `TIMESTAMP` приводятся к `TRUE`

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SELECT CAST(now() AS BOOLEAN)
"

#### Приведение значений `BOOLEAN` к другим типам данных

Логические значения можно приводить к другим типам:

In [ ]:
impala-shell -i "${IMPALA_HOST}" --vertical -q "
SELECT CAST(TRUE AS INT) true_int
     , CAST(FALSE AS TINYINT) false_tyniint
     , CAST(TRUE AS DOUBLE) true_double
     , CAST(FALSE AS STRING) false_string
     , CAST(TRUE AS TIMESTAMP) true_ts
     , CAST(FALSE AS TIMESTAMP) false_ts
"

#### Булевые операции

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SELECT TRUE AND FALSE t_a_f
     , TRUE OR FALSE t_o_f
"

## Тип данных `ARRAY`

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
CREATE TABLE movies (
    name STRING,
    tags ARRAY <STRING>
)
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
DESC EXTENDED movies
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
DESC movies.tags
"

Невозможно создать значение типа `ARRAY` в Impala:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
INSERT INTO movies (name, tags)
VALUES ('Interstellar', ARRAY('sci-fi', 'outer-space', 'drama'))
" || true

К счастью можно использовать Apache Hive для наполнения таблицы:

In [ ]:
HOST=hive \
new_file /tmp/query.sql <<EOF
INSERT INTO my_db.movies (name, tags)
VALUES ('Interstellar', ARRAY('sci-fi', 'outer-space', 'drama'));

SELECT * FROM my_db.movies;
EOF

beeline -u "jdbc:hive2://${HIVE_HOST}:10000/" -f /tmp/query.sql

Данные недоступны:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
SELECT *
  FROM movies
"

При обновлении данных в таблице через внешнюю систему, такую как Apache Hive, например, необходимо явно обновить таблицу:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
REFRESH movies;

SELECT *
  FROM movies
"

Поле типа `ARRAY` недоступно. Для получения данные из поля типа `ARRAY` необходимо явно ссылаться на поле типа `ARRAY` в разделе `FROM`:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
SELECT name
     , tags.pos
     , tags.item
  FROM movies
     , movies.tags
"

## Тип данных `STRUCT`

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
CREATE TABLE songs (
    name STRING,
    creaters STRUCT<
        composer: STRING,
        singer: STRING,
        writer: STRING,
        label: STRING
    >
)
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
DESC EXTENDED songs
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
DESC songs.creaters
"

Невозможно создать значение типа `STRUCT` в Impala:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
INSERT INTO songs (name, creaters)
VALUES ('Africa', NAMED_STRUCT('composer', 'David Paich', 'singer', 'Toto', 'writer', 'Toto','label', 'Columbia'))
" || true

In [ ]:
HOST=hive \
new_file /tmp/query.sql <<EOF
INSERT INTO my_db.songs (name, creaters)
VALUES ('Africa', NAMED_STRUCT('composer', 'David Paich', 'singer', 'Toto', 'writer', 'Toto','label', 'Columbia'));

SELECT * FROM my_db.songs;
EOF

beeline -u "jdbc:hive2://${HIVE_HOST}:10000/" -f /tmp/query.sql

Данные недоступны:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
SELECT *
  FROM songs
"

При обновлении данных в таблице через внешнюю систему, такую как Apache Hive, например, необходимо явно обновить таблицу:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
REFRESH songs;

SELECT *
  FROM songs
"

Поле типа `STRUCT` недоступно. Для получения данных из поля типа `STRUCT` необходимо явно ссылаться на поле из колонки типа `STRUCT`:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
SELECT name
     , creaters.composer
     , creaters.singer
     , creaters.writer
     , creaters.label
  FROM songs
"

## Тип данных `MAP`

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
DROP TABLE IF EXISTS box_office
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
CREATE TABLE box_office (
    name STRING,
    earnings MAP<STRING, DECIMAL(20, 3)>
)
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
DESC EXTENDED box_office
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
DESC box_office.earnings
"

Невозможно создать значение типа `MAP` в Impala:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
INSERT INTO box_office (name, earnings)
VALUES ('Titanic', MAP('US Gross', 600683057, 'World Wide Gross', 1843373318))
" || true

На выручку приходит Apache Hive:

In [ ]:
HOST=hive \
new_file /tmp/query.sql <<EOF
INSERT INTO my_db.box_office (name, earnings)
SELECT 'Titanic'
     , MAP(
         'US Gross', CAST(600683057 AS DECIMAL(20, 3)),
         'World Wide Gross', CAST(1843373318 AS DECIMAL(20, 3))
       );

SELECT name
     , earnings['US Gross'] us
     , earnings['World Wide Gross'] ww
  FROM my_db.box_office;
EOF

beeline -u "jdbc:hive2://${HIVE_HOST}:10000/" -f /tmp/query.sql

Данные, добавленные через внешние системы по прежнему недоступны сразу:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
SELECT *
  FROM box_office
"

Необходимо выполнить `REFRESH`:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
REFRESH box_office;

SELECT *
  FROM box_office
"

Поле типа `MAP` недоступно по умолчанию. Для доступа к значениям внутри поля типа `MAP` необходимо обращаться к полю типа `MAP` в `FROM` секции при запросе:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
SELECT name
     , earnings.key
     , earnings.value
  FROM box_office
     , box_office.earnings
"

## Вывод

1. Impala предлагает стандартные примитивные типы данных, аналогичные другим базам данных;
1. Impala имеет составные типы данных: `ARRAY`, `MAP`, `STRUCT`, доступные только на чтение;
1. Impala предлагает достаточно гибкую систему приведения типов.

## Задания

1. Написать запрос `SELECT`, который выводит ваши фамилию, имя и отчество (ФИО) в одную строку на кириллице и латинице. Какой тип данных будет иметь такая строка?
1. Разбить ФИО на 3 слова при помощи функций для работы со строками. Выполнить для латиницы и кириллицы.
1. Привести ФИО к типу `CHAR(7)`. Какой эффект наблюдается?
1. Привести ФИО к типу `VARCHAR(50)`;
1. Извлечь квадратный корень из длины вашего ФИО. Длину считать по символам;
1. Возвести число пи в 4 степень;
1. Написать запрос `SELECT`, который выводит день вашего рождения;
1. Привести день вашего рождения к типу `TIMESTAMP`;
1. Какая дата получится, если к ближайшей дате 29 февраля (високосный год) добавить 1 год?

### Ответы

1. Написать запрос `SELECT`, который выводит ваши фамилию, имя и отчество (ФИО) в одную строку на кириллице и латинице. Какой тип данных будет иметь такая строка?

<details>
    <summary>Ответ</summary>

Вывод ФИО:
```sql
SELECT 'Никонов Ермолай Лаврентьевич' cyrillic
     , 'Nikonov Ermolay Lavrentyevich' latin
```
Автоматически назначается тип `STRING` (обратите внимание на ошибку, которая указывает из какого типа выполняется конвертация):
 
```sql
SELECT CAST('Никонов Ермолай Лаврентьевич' AS DATE)
```
</details>

2. Разбить ФИО на 3 слова при помощи функций для работы со строками. Выполнить для латиницы и кириллицы.

<details>
    <summary>Ответ</summary>

```sql
WITH names (name) AS (
    SELECT 'Никонов Ермолай Лаврентьевич' UNION
    SELECT 'Nikonov Ermolay Lavrentyevich'
)
SELECT SUBSTR(name, 1, INSTR(name, ' ')) lastname
     , SUBSTR(name, INSTR(name, ' ') + 1, INSTR(name, ' ', 1, 2) - INSTR(name, ' ')) firstname
     , SUBSTR(name, INSTR(name, ' ', 1, 2) + 1) paternalname
  FROM names
```
</details>

3. Привести ФИО к типу `CHAR(7)`. Какой эффект наблюдается?

<details>
    <summary>Ответ</summary>

Только часть ФИО остается доступным, причем имя на кириллице обрежается по середине четвертого символа, т.к. кириллический символ занимает 2 байта:

```sql
WITH names (name) AS (
    SELECT 'Никонов Ермолай Лаврентьевич' UNION
    SELECT 'Nikonov Ermolay Lavrentyevich'
)
SELECT CAST(name AS CHAR(7))
  FROM names
```
</details>

4. Привести ФИО к типу `VARCHAR(50)`;

<details>
    <summary>Ответ</summary>

Если значение меньше, чем размер выделенного буфера, то будет занято ровно столько места, сколько необходимо для размещения значения целиком. Если значение длинее 50 байт, то значение обрежется:

```sql
WITH names (name) AS (
    SELECT 'Никонов Ермолай Лаврентьевич' UNION
    SELECT 'Nikonov Ermolay Lavrentyevich'
), names_vchar50 (name) AS (
SELECT CAST(name AS VARCHAR(50))
  FROM names
)
SELECT name, LENGTH(name), CHAR_LENGTH(name)
  FROM names_vchar50
```
</details>

5. Извлечь квадратный корень из длины вашего ФИО. Длину считать по символам;

<details>
    <summary>Ответ</summary>

Необходимо активировать режим UTF8, для подсчета длины ФИО по символам:

```bash
impala-shell -i "${IMPALA_HOST}" -d my_db -q "
SET UTF8_MODE=true;

WITH names (name) AS (
    SELECT 'Никонов Ермолай Лаврентьевич' UNION
    SELECT 'Nikonov Ermolay Lavrentyevich'
)
SELECT name, LENGTH(name) len, SQRT(LENGTH(name)) sqrt
  FROM names
"
```
</details>

6. Возвести число пи в 4 степень;

<details>
    <summary>Ответ</summary>

```sql
WITH pi (value) AS (
    SELECT atan(1) * 4
)
SELECT value
     , POW(value, 4)
  FROM pi
```
</details>

7. Написать запрос `SELECT`, который выводит день вашего рождения;

<details>
    <summary>Ответ</summary>

```sql
SELECT DATE '1989-01-06' dob
```
</details>

8. Привести день вашего рождения к типу `TIMESTAMP`;

<details>
    <summary>Ответ</summary>

```sql
WITH dob(value) AS (
    SELECT DATE '1989-01-06'
)
SELECT CAST(value AS TIMESTAMP) dob_ts from dob
```
</details>

9. Какая дата получится, если к ближайшей дате 29 февраля (високосный год) добавить 1 год?

<details>
    <summary>Ответ</summary>

```sql
WITH leap_year(leap_day) AS (
    SELECT DATE '2024-02-29'
)
SELECT leap_day
     , ADD_MONTHS(leap_day, 12) leap_day_next_year
  FROM leap_year
```
</details>